<a href="https://colab.research.google.com/github/shruti-singh/PaperAcceptancePrediction/blob/master/zeel/refextract/refextract_parallel_FromURLs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y python-magic
!sudo apt-get install -y xpdf
!pip install refextract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libmagic-mgc libmagic1
Suggested packages:
  file
The following NEW packages will be installed:
  libmagic-mgc libmagic1 python-magic
0 upgraded, 3 newly installed, 0 to remove and 25 not upgraded.
Need to get 262 kB of archives.
After this operation, 5,255 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.3 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.3 [68.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-magic all 2:0.4.15-1 [9,192 B]
Fetched 262 kB in 1s (304 kB/s)
Selecting previously unselected package libmagic-mgc.
(Reading database ... 133872 files and directories currently installed.)
Preparing to unpack .../libmagic-mgc_1%3a5.32-2ubuntu0.3_amd64.deb ...


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import refextract
#from tika import parser
from glob import glob
import pandas as pd
from time import time
import pickle
import multiprocessing as mp
import psutil
import os
import sys
from pprint import pprint
from refextract import extract_references_from_url
import shutil
from time import sleep

In [4]:
# Testing single output (It may take a few seconds)
forum = 'HJGv1Z-AW'
references = extract_references_from_url('https://openreview.net/pdf?id='+forum)
pprint(references)

[{'misc': ['Jacob Andreas and Dan Klein. Analogs of linguistic structure in '
           'deep representations. arXiv preprint'],
  'raw_ref': ['Jacob Andreas and Dan Klein. Analogs of linguistic structure in '
              'deep representations. arXiv preprint arXiv:1707.08139, 2017.'],
  'reportnumber': ['arXiv:1707.08139'],
  'year': ['2017']},
 {'misc': ['Trapit Bansal, Jakub Pachocki, Szymon Sidor, Ilya Sutskever, and '
           'Igor Mordatch. Emergent complexity via multi-agent competition. '
           'arXiv preprint'],
  'raw_ref': ['Trapit Bansal, Jakub Pachocki, Szymon Sidor, Ilya Sutskever, '
              'and Igor Mordatch. Emergent complexity via multi-agent '
              'competition. arXiv preprint arXiv:1710.03748, 2017.'],
  'reportnumber': ['arXiv:1710.03748'],
  'year': ['2017']},
 {'misc': ['John Batali. Computational simulations of the emergence of '
           'grammar. pp. 405-426'],
  'raw_ref': ['John Batali. Computational simulations of the emergence o

In [9]:
# Loading data to load forum ids
data = pd.read_pickle('/content/drive/My Drive/SEM-1 ML Project/data/data_csv-2017-20.pkl')
data['year_z'].value_counts()

2020    1374
2019    1004
2018     672
2017     396
Name: year_z, dtype: int64

In [0]:
def extract(forum):
  path = '/content/drive/My Drive/Colab Notebooks/ML Project/data/papers/refall/'+forum
  try:
    if not os.path.exists(path):
      with open(path,'wb') as f:
        pickle.dump(refextract.extract_references_from_url('https://openreview.net/pdf?id='+forum), f)
  except RemoteTraceback:
    print(forum)

In [11]:
print('N CPU =', psutil.cpu_count())
workers = mp.Pool(psutil.cpu_count())

N CPU = 2


In [0]:
# Extracting references parallally 
#(Run this script on powerfull servers to take advantage of parallelization. )
init = time()
workers.map(extract, data.index)
print(time() - init)

0.5053751468658447


In [13]:
# testing if all files are downloaded (if not, run the script again or troubleshoot)
path = '/content/drive/My Drive/SEM-1 ML Project/data/papers/refall/*'
files = glob(path)
print(data['year_z'].value_counts().sum(),' == ',len(files))

3446  ==  3440


In [16]:
# Loading and Verifying if all objects are at place
ref_obs = glob('/content/drive/My Drive/SEM-1 ML Project/data/papers/refall/*')
clean_path = '/content/drive/My Drive/SEM-1 ML Project/data/papers/refclean/'
len(ref_obs)

3440

In [17]:
def validate(ref):
  if 'year' in ref.keys():
    if 'Published as a conference paper at ICLR' not in ref['raw_ref']:
      if int(ref['year'][0]) in range(1800,2021):
        return True
  return False

i = 0
library = {}
for obj in ref_obs:
  try:
    refs = pd.read_pickle(obj)
  except EOFError:
    os.remove(obj)
  library.update({obj.split('/')[-1]: [ref['year'][0] for ref in refs if validate(ref)]})
  print(i)
  i += 1
  #sleep(0.01)
len(library)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219


KeyboardInterrupt: ignored

In [19]:
ref_df = pd.DataFrame()
ref_df['forum'] = library.keys()
ref_df['ref_years'] = [[int(i) for i in j] for j in library.values()]
ref_df.set_index('forum', inplace=True)
print(ref_df.head(2))
print('\n\n')

# Preprocessing
ref_df['ref_ylen'] = ref_df['ref_years'].apply(len) # Number of refs
ref_df['ref_latest'] = ref_df['ref_years'].apply(lambda x: max(x+[1992])) #latest ref
print(ref_df['ref_latest'].value_counts())
print('\n\n')
print(ref_df.head(2))

                                                    ref_years
forum                                                        
SkVhlh09tX  [2017, 2016, 2015, 2018, 2013, 2018, 2015, 201...
Hyl7ygStwB  [2015, 2019, 2019, 2014, 2014, 2019, 2015, 201...



2019    142
2018     61
2020      9
2017      6
2015      1
2009      1
Name: ref_latest, dtype: int64



                                                    ref_years  ...  ref_latest
forum                                                          ...            
SkVhlh09tX  [2017, 2016, 2015, 2018, 2013, 2018, 2015, 201...  ...        2019
Hyl7ygStwB  [2015, 2019, 2019, 2014, 2014, 2019, 2015, 201...  ...        2019

[2 rows x 3 columns]


In [0]:
# Assigning these features to original csv
data['ref_latest'] = ref_df['ref_latest']
data['ref_len'] = ref_df['ref_ylen']
data['ref_years'] = ref_df['ref_years']

In [0]:
# Saving the updated csv
data_csv.to_pickle('/content/drive/My Drive/Colab Notebooks/ML Project/data/data_csv_refextracted-2017-20.pkl')